In [8]:
import mediapipe as mp
import os 
import cv2 
from keras.models import load_model 
from PIL import Image,ImageDraw,ImageFont
import numpy as np

In [20]:
model = load_model('./model2_3_110.h5')

In [2]:
mp_hands = mp.solutions.hands 
hands = mp_hands.Hands(static_image_mode=True,min_detection_confidence=0.3,max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [22]:
label = ['अ','आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ओ', 'औ','क' , 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ','ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ','ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह','क्ष', 'त्र', 'ज्ञ', 'श्र','्','्र','blank']

In [3]:
def draw_square(frame,results,SQUARE_SIZE):
    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]
        
        x = hand_landmarks.landmark[9].x * frame.shape[1]
        y = hand_landmarks.landmark[9].y * frame.shape[0]
        
        w = hand_landmarks.landmark[12].x * frame.shape[1] - x 
        h = hand_landmarks.landmark[12].y * frame.shape[0] - y
    
        x_center = int(x )  
        y_center = int(y)
        
        x1 = int(x_center - SQUARE_SIZE/2)
        y1 = int(y_center - SQUARE_SIZE/2)
        
        cv2.rectangle(frame, (x1, y1), (x1 + SQUARE_SIZE, y1 + SQUARE_SIZE), 
                      (0, 255, 0), 2)

In [4]:
def crop_image(results,frame,SQUARE_SIZE):
    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]
        
        x = hand_landmarks.landmark[9].x * frame.shape[1]
        y = hand_landmarks.landmark[9].y * frame.shape[0]
        
        w = hand_landmarks.landmark[12].x * frame.shape[1] - x
        h = hand_landmarks.landmark[12].y * frame.shape[0] -y 
        
        x_center = int(x)
        y_center = int(y)
        
        x1 = int(x_center - SQUARE_SIZE/2)
        y1= int(y_center - SQUARE_SIZE/2)
        cv2.rectangle(frame, (x1, y1), (x1 + SQUARE_SIZE, y1 + SQUARE_SIZE), 
                      (0, 255, 0), 2)
        return (x1,y1)

In [5]:
def create_hand_landmark(results,mask):
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(mask,
                                      hand_landmarks,
                                      mp_hands.HAND_CONNECTIONS,
                                      mp_drawing_styles.DrawingSpec(color=(255,0,255),thickness=1,circle_radius=1),
                                      mp_drawing_styles.DrawingSpec(color=(255,0,255),thickness=1,circle_radius=1))
    return mask

In [21]:
def extract_features(image):
    feature = np.array(image)
    feature = feature.reshape(1,200,200,1)
    return feature/255.0

In [26]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,540)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,540)
SQUARE_SIZE =200
while cap.isOpened():
    ret,frame = cap.read()
    image = frame.copy()
    results = hands.process(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
    draw_square(frame,results,SQUARE_SIZE)
    try:
        (x1,y1) = crop_image(results,frame,SQUARE_SIZE)
        cropped_image = image[y1:y1+SQUARE_SIZE,x1:x1+SQUARE_SIZE]
        img1 = cv2.cvtColor(cropped_image,cv2.COLOR_BGR2RGB)
        results = hands.process(img1)
        mask = np.zeros_like(img1) #create blank mask
        mask = create_hand_landmark(results,mask)
        masked_img = cv2.bitwise_and(img1,mask)
        masked_img= cv2.cvtColor(masked_img,cv2.COLOR_RGB2GRAY)
        test_image = masked_img.copy()
        test_image = extract_features(test_image)
        pred  = model.predict(test_image)
        prediction_label = label[pred.argmax()]
        accu = "{:.2f}".format(np.max(pred)*100)
        # print(accu)
        # if accu>80:
        print(prediction_label,accu)
        # masked_img = cv2.IMREAD_GRAYSCALE(masked_img)
        cv2.imshow('frame1',masked_img)
    except Exception as e:
        pass
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 33ms/step
ज्ञ 99.70
1/1 [==============================] - 0s 35ms/step
ज्ञ 99.70
1/1 [==============================] - 0s 30ms/step
ज्ञ 99.70
1/1 [==============================] - 0s 31ms/step
य 100.00
1/1 [==============================] - 0s 31ms/step
इ 97.98
1/1 [==============================] - 0s 30ms/step
क्ष 100.00
1/1 [==============================] - 0s 31ms/step
क्ष 100.00
1/1 [==============================] - 0s 36ms/step
क्ष 100.00
1/1 [==============================] - 0s 31ms/step
क्ष 100.00
1/1 [==============================] - 0s 31ms/step
क्ष 100.00
1/1 [==============================] - 0s 32ms/step
फ 100.00
1/1 [==============================] - 0s 29ms/step
ञ 100.00
1/1 [==============================] - 0s 32ms/step
क्ष 100.00
1/1 [==============================] - 0s 28ms/step
ञ 100.00
1/1 [==============================] - 0s 30ms/step
ञ 100.00
1/1 [==============================] - 0s 30ms/step
ण 100.00
1/1 [=====

In [16]:
cap.release()
cv2.destroyAllWindows()